In [35]:
import os
import pandas as pd
import copy

os.chdir("/home/tales/dev/master/mdc_analysis/")

from src.taxonomy.category_mapping import CategoryMapper
from src.poi_grabber import google_places
from src.utils.others import remove_list_elements
from src.exceptions.exceptions import NoCategoryMatched, NotValidTypes

### Loading

In [2]:
categ_mapper = CategoryMapper()

In [3]:
def load_sr_timestamp_quartiles():
    return pd.read_csv("outputs/sr_time_quartiles/sr_time_quartiles.csv")

In [4]:
sr_time_quartiles = load_sr_timestamp_quartiles()
print(len(sr_time_quartiles))
sr_time_quartiles.sample(10)

25651


,user_id,place_id,q1,q2,q3,sr_list
10800,5964,ChIJ7XMyQ9atj0cRnyB_tIjL7NQ,32.200000,32.200000,32.200000,['5964_284']
15141,6026,ChIJ6cFerhubjkcRhNWk7xYRekU,5.133333,5.133333,5.133333,['6026_336']
14148,6072,ChIJ1UYK32FujkcRTgqQaYvM_QY,28.925000,29.250000,29.575000,"['6072_485', '6072_488']"
16254,5949,ChIJpSkJgzMujEcR5gGkCbRli2E,670.150000,670.150000,670.150000,['5949_2']
15092,6077,ChIJIcsdMkQxjEcROBG9bRB4uyA,14.916667,14.916667,14.916667,['6077_67']
10656,6001,ChIJk7eZMg4xjEcRjyJMnE-rnMI,17.550000,17.550000,17.550000,['6001_1339']
23558,5944,ChIJOX1gOHM3jEcRUhRTwpa6E4E,13.850000,13.850000,13.850000,['5944_396']
4634,6083,ChIJSUiPuKExjEcRM5b3rQr1imI,28.883333,28.883333,28.883333,['6083_181']
8509,6192,ChIJ4TchpSyDjkcRY1yMUYfCLkk,12.100000,12.100000,12.100000,['6192_310']
3944,6175,ChIJX2WSGdKcjkcRDAwrJ_s_IXM,8.050000,8.050000,8.050000,['6175_451']


In [5]:
pois = google_places.load_all_google_places_data()
pois[["place_id", "types"]].head(5)

,place_id,types
place_id,,
ChIJTcpEIDQujEcRfRIRizbvj4k,ChIJTcpEIDQujEcRfRIRizbvj4k,"[accounting, finance, point_of_interest, local..."
ChIJPV-d_9dkjEcRh-AFXijs5H0,ChIJPV-d_9dkjEcRh-AFXijs5H0,"[store, point_of_interest, establishment]"
ChIJ6TtT3J1EjEcRdmsz6L2gHTU,ChIJ6TtT3J1EjEcRdmsz6L2gHTU,"[beauty_salon, point_of_interest, establishment]"
ChIJLZi4AC8ujEcRvSGJiRYVAk0,ChIJLZi4AC8ujEcRvSGJiRYVAk0,"[health, point_of_interest, establishment]"
ChIJKTbDs6ygmkcRhp_reVwKcjc,ChIJKTbDs6ygmkcRhp_reVwKcjc,"[finance, point_of_interest, establishment]"


### Cleaning

In [6]:
pois["types"] = pois["types"].apply(lambda type_list : remove_list_elements(type_list, ["point_of_interest", "establishment", "parking"]))

In [7]:
print(len(pois[["place_id", "types"]]))
pois[["place_id", "types"]].sample(8)

41739


,place_id,types
place_id,,
ChIJ6Wh5bzgujEcR0i_v-VPwzWo,ChIJ6Wh5bzgujEcR0i_v-VPwzWo,[lodging]
ChIJRSPe-CdljEcRvQUbL4LILao,ChIJRSPe-CdljEcRvQUbL4LILao,[beauty_salon]
ChIJ3SAF8cOZjkcRDnxQlZWSH64,ChIJ3SAF8cOZjkcRDnxQlZWSH64,[store]
ChIJ61c599UxjEcRHB_p6KRLMHw,ChIJ61c599UxjEcRHB_p6KRLMHw,"[doctor, health]"
ChIJ8RFCavcthEcR4PihLiEEjO8,ChIJ8RFCavcthEcR4PihLiEEjO8,"[locality, political]"
ChIJecU-V9EvjEcRsfjv9I5Bl-g,ChIJecU-V9EvjEcRsfjv9I5Bl-g,"[bus_station, transit_station]"
ChIJjwQNzeAwjEcRU_rUX28nQoA,ChIJjwQNzeAwjEcRU_rUX28nQoA,"[shoe_store, store]"
ChIJcQYpmxpjjEcROb48TGKoOxc,ChIJcQYpmxpjjEcROb48TGKoOxc,"[doctor, health]"


In [8]:
pois = pois[pois["types"].apply(len) > 0]
pois[["place_id", "types"]].sample(10)

,place_id,types
place_id,,
ChIJWYkJ_Y-kj0cRHXhloe5heDY,ChIJWYkJ_Y-kj0cRHXhloe5heDY,"[liquor_store, convenience_store, restaurant, ..."
ChIJccKSw9adjkcRFFFh3egWKxg,ChIJccKSw9adjkcRFFFh3egWKxg,"[doctor, health]"
ChIJwx0PLX2uj0cRmus8Gtik-_U,ChIJwx0PLX2uj0cRmus8Gtik-_U,[health]
ChIJTyAqVKy5kUcRBStnwgl-0yc,ChIJTyAqVKy5kUcRBStnwgl-0yc,"[night_club, bar]"
ChIJaYEHW1QjjEcR5Wh565q3Z8o,ChIJaYEHW1QjjEcR5Wh565q3Z8o,"[real_estate_agency, finance]"
ChIJeyOIXz7TkUcROqY-Drn4cs4,ChIJeyOIXz7TkUcROqY-Drn4cs4,"[restaurant, food]"
ChIJG-LOYggKkEcR4yyloMCNMLk,ChIJG-LOYggKkEcR4yyloMCNMLk,"[shoe_store, store]"
ChIJGZL8OdVJjkcReAQrVoCrAKI,ChIJGZL8OdVJjkcReAQrVoCrAKI,"[dentist, health]"
ChIJAQAAANW3hUcRir_dsC-mpXA,ChIJAQAAANW3hUcRir_dsC-mpXA,"[church, place_of_worship]"


### Categorization 
0.0.categ_v1

In [9]:
categs = []

for types in pois["types"]:

    try:
        categ = categ_mapper.map_categ(types, method="most_specific")
        categs.append(categ)

    except NotValidTypes:
        categs.append("NotValidTypes")

    except NoCategoryMatched:
        categs.append("NoCategoryMatched")

In [10]:
pois["categ"] = categs

/home/tales/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [11]:
pois[["place_id", "types", "categ"]].head(8)

,place_id,types,categ
place_id,,,
ChIJTcpEIDQujEcRfRIRizbvj4k,ChIJTcpEIDQujEcRfRIRizbvj4k,"[accounting, finance, local_government_office]",finance
ChIJPV-d_9dkjEcRh-AFXijs5H0,ChIJPV-d_9dkjEcRh-AFXijs5H0,[store],store
ChIJ6TtT3J1EjEcRdmsz6L2gHTU,ChIJ6TtT3J1EjEcRdmsz6L2gHTU,[beauty_salon],beauty_salon
ChIJLZi4AC8ujEcRvSGJiRYVAk0,ChIJLZi4AC8ujEcRvSGJiRYVAk0,[health],health
ChIJKTbDs6ygmkcRhp_reVwKcjc,ChIJKTbDs6ygmkcRhp_reVwKcjc,[finance],finance
ChIJlV3lSSQKjkcR4TT3Ge3RTpc,ChIJlV3lSSQKjkcR4TT3Ge3RTpc,"[cafe, food]",cafe
ChIJB63ZuK25kUcRLGzhM7CLiX4,ChIJB63ZuK25kUcRLGzhM7CLiX4,"[jewelry_store, store]",store
ChIJzaP4gsZujkcRTffguoQ00Dk,ChIJzaP4gsZujkcRTffguoQ00Dk,"[gym, health]",health


### Adding sr_time_quartiles Category data

In [12]:
sr_time_quartiles = sr_time_quartiles.merge(pois[["place_id", "categ"]], on="place_id", how="inner")
sr_time_quartiles.sample(8)

/home/tales/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: FutureWarning: 'place_id' is both an index level and a column label.
Defaulting to column, but this will raise an ambiguity error in a future version
  """Entry point for launching an IPython kernel.


,user_id,place_id,q1,q2,q3,sr_list,categ
2117,6167,ChIJ8XO9VioujEcRbsMHI-Mg8DQ,6.500000,6.500000,6.500000,['6167_387'],health
11744,5959,ChIJiSh1HBgujEcRcKSDSMSCNx0,21.116667,21.116667,21.116667,['5959_1280'],NoCategoryMatched
11325,6015,ChIJ6ef5Nwofj0cRvmL6NLvE9bo,35.883333,35.883333,35.883333,['6015_55'],real_estate_agency
7864,5967,ChIJNx0ZvGC8jkcR5Fc7FUZHUkI,5.316667,5.316667,5.316667,['5967_361'],store
1360,6027,ChIJN0NwRw4xjEcRNydt-U4lugg,41.050000,41.050000,41.050000,['6027_334'],store
5861,5947,ChIJ250FqA4fj0cRQ77qdx1LCns,24.066667,24.066667,24.066667,['5947_1315'],restaurant
12089,5953,ChIJJQ39QNovjEcR4DpkyAUcqyk,5.033333,5.033333,5.033333,['5953_80'],travel_agency
1447,6178,ChIJSxdgUSeajkcR9NbOzps4TzE,28.116667,28.116667,28.116667,['6178_1149'],restaurant


In [13]:
sr_time_quartiles.groupby(["user_id", "categ"]).apply(lambda group : group["q2"].mean())

user_id  categ                  
5924     NoCategoryMatched           30.711111
         bakery                      14.239286
         bar                          8.589583
         beauty_salon                34.662500
         cafe                        29.216667
         convenience_store            9.466667
         finance                     38.996569
         general_contractor          59.258333
         grocery_or_supermarket      13.920833
         hair_care                  147.307143
         health                     323.368571
         lawyer                      41.772222
         liquor_store                 7.550000
         local_government_office     11.316667
         lodging                     38.804167
         place_of_worship             5.250000
         real_estate_agency          40.641667
         restaurant                  61.727244
         school                      22.991667
         store                       43.076724
         supermarket       

In [14]:
sr_time_quartiles.groupby(["user_id", "categ"]).apply(lambda group : group["categ"].value_counts())

user_id  categ                                           
5924     NoCategoryMatched        NoCategoryMatched           6
         bakery                   bakery                      7
         bar                      bar                         4
         beauty_salon             beauty_salon                4
         cafe                     cafe                        5
         convenience_store        convenience_store           1
         finance                  finance                    17
         general_contractor       general_contractor          2
         grocery_or_supermarket   grocery_or_supermarket      4
         hair_care                hair_care                   7
         health                   health                     35
         lawyer                   lawyer                      3
         liquor_store             liquor_store                2
         local_government_office  local_government_office     2
         lodging                  lodging     

In [15]:
place_freq = sr_time_quartiles.groupby(["user_id", "categ"]).apply(lambda group : group["categ"].value_counts()).to_frame()
place_freq = place_freq.rename({"categ": "freq"}, axis=1).reset_index()
del place_freq["level_2"]
place_freq.sample(8)

,user_id,categ,freq
711,5972,NoCategoryMatched,8
434,5956,beauty_salon,2
2063,6079,supermarket,2
1536,6036,liquor_store,2
2027,6076,liquor_store,3
141,5937,supermarket,1
1202,6010,real_estate_agency,1
507,5960,local_government_office,1


In [17]:
len(place_freq), len(place_freq[["user_id", "categ"]].drop_duplicates())

(2654, 2654)

#### Fill not visited places with freq = 0

In [27]:
complete_place_freq = []

for user_id in place_freq["user_id"].drop_duplicates():
    user_data = place_freq[place_freq["user_id"] == user_id]
        
    for categ in place_freq["categ"].drop_duplicates():
        row = {}
        row["user_id"] = user_id
        row["categ"] = categ
        
        if categ in user_data["categ"].tolist():
            row["freq"] = user_data[user_data["categ"] == categ]["freq"].item()
        else:
            row["freq"] = 0
            
        complete_place_freq.append(row)

place_freq = pd.DataFrame(complete_place_freq)[["user_id", "categ", "freq"]]

In [29]:
place_freq.sample(8)

,user_id,categ,freq
3229,6096,cafe,0
1250,5986,NoCategoryMatched,5
2402,6053,bar,1
2853,6073,beauty_salon,0
234,5940,hair_care,3
2740,6068,place_of_worship,5
3903,6189,beauty_salon,4
816,5964,real_estate_agency,5


#### Normalization by user

In [30]:
def normalize(series):
    amplitude = series.max() - series.min()
    return (series - series.min()) / amplitude

In [41]:
place_freq_norm = pd.DataFrame()

for user_id in place_freq["user_id"].drop_duplicates():
    user_data = copy.deepcopy(place_freq[place_freq["user_id"] == user_id])
    user_data["freq_norm"] = normalize(user_data["freq"])

    place_freq_norm = place_freq_norm.append(user_data)
    
place_freq_norm.head(10)

,user_id,categ,freq,freq_norm
0,5924,NoCategoryMatched,6,0.171429
1,5924,bakery,7,0.200000
2,5924,bar,4,0.114286
3,5924,beauty_salon,4,0.114286
4,5924,cafe,5,0.142857
5,5924,convenience_store,1,0.028571
6,5924,finance,17,0.485714
7,5924,general_contractor,2,0.057143
8,5924,grocery_or_supermarket,4,0.114286
9,5924,hair_care,7,0.200000


### Clusterization

In [42]:
from sklearn.cluster import KMeans

In [61]:
wide_place_freq = []

for user_id in place_freq_norm["user_id"].drop_duplicates():
    user_data = copy.deepcopy(place_freq_norm[place_freq_norm["user_id"] == user_id])
    
    row = {"user_id": user_id}
    
    for categ in user_data["categ"]:
        row[categ] = user_data[user_data["categ"] == categ]["freq"].item()
    
    wide_place_freq.append(row)
    
wide_place_freq = pd.DataFrame(wide_place_freq)
wide_place_freq.head()

,NoCategoryMatched,NotValidTypes,bakery,bar,beauty_salon,cafe,convenience_store,finance,general_contractor,grocery_or_supermarket,...,place_of_worship,political,real_estate_agency,restaurant,school,store,supermarket,transit_station,travel_agency,user_id
0,6,0,7,4,4,5,1,17,2,4,...,1,0,12,26,4,29,1,9,4,5924
1,5,0,1,2,3,2,1,6,2,3,...,2,2,12,24,2,36,3,8,2,5925
2,2,0,1,0,0,4,5,16,0,3,...,6,0,6,20,1,33,1,14,0,5927
3,7,0,1,3,5,7,1,11,3,1,...,2,0,2,27,4,46,5,20,1,5928
4,1,0,1,7,5,4,1,5,3,4,...,2,0,2,24,0,29,1,12,4,5935


In [62]:
print(len(wide_place_freq) / 5.)
user_ids = wide_place_freq["user_id"].to_frame()
del wide_place_freq["user_id"]

kmeans = KMeans(n_clusters=25, random_state=0).fit(wide_place_freq)

32.6


In [63]:
user_ids["cluster"] = kmeans.labels_

In [65]:
user_ids

,user_id,cluster
0,5924,9
1,5925,14
2,5927,13
3,5928,1
4,5935,8
5,5936,10
6,5937,22
7,5938,8
8,5939,8
9,5940,2


In [66]:
user_ids.to_csv("outputs/sr_time_quartiles/freq_clusters.csv", index=False)